<a href="https://colab.research.google.com/github/adigenova/uohdb/blob/main/code/NoSQL_ArangoDB_ejemplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![arangodb](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/ArangoDB_logo.png?raw=1)

# Tutorial de  Arango DB

ArangoDB es una base de datos multimodelo  con modelos de datos flexibles para documentos, grafos y clave/valor. Posee el lenguaje AQL que es similar al SQL para realizar consultas.

# Instalación

Antes de comenzar con ArangoDB, debemos preparar nuestro entorno y crear una base de datos temporal en el Servicio Oasis.

In [1]:
%%capture
!git clone -b oasis_connector --single-branch https://github.com/arangodb/interactive_tutorials.git
!rsync -av interactive_tutorials/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [2]:
import json
import requests
import sys
import oasis
import time

from pyArango.connection import *
from arango import ArangoClient

Creamos una base de datos temporal

In [ ]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="AQLCrudTutorial", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')
# Connect to the temp database
conn = oasis.connect(login)
pyAr_db = conn[login["dbName"]]

In [4]:
python_arango_db = oasis.connect_python_arango(login)
aql = python_arango_db.aql

In [ ]:
print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

Podemos usar la URL anterior para revisar nuestra base de datos arango temporal.



## Creando Colecciones

Antes de que podamos insertar documentos con AQL, necesitamos un lugar para colocarlos: una colección

In [6]:
# creamos la coleccion usando pyArango
pyAr_db.createCollection(name="Personajes")
if python_arango_db.has_collection("Personajes"):
  pass
else:
  python_arango_db.create_collection("Pesonajes")

##  Creando y leyendo documentos

La sintaxis para crear un nuevo documento es INSERT document INTO collectionName. El documento es un objeto JSON, que se compone de pares de clave de atributo y valor.

Las claves son siempre secuencias de caracteres (cadenas), mientras que los valores de los atributos pueden tener diferentes tipos:

* null
* boolean (true, false)
* number (integer and floating point)
* string
* array
* object


In [7]:
insert_query = """
INSERT {
    "nombre": "Alex",
    "apellido": "Di Genova",
    "vivo": true,
    "edad": 41,
    "caracteres": ["A","H","C","N","P"]
} INTO Personajes
"""
# python-arango
aql.execute(insert_query)

<Cursor>

Chequamos si la inserción se realizo con exito en la colleción Personajes.
Para cada documento de la colección, a c se le asigna un documento, que luego se devuelve según el cuerpo del bucle.

In [ ]:
todos_personajes = """
FOR c IN Personajes
    RETURN c
"""
query_result = aql.execute(todos_personajes)
for doc in  query_result:
 print(doc)

In [9]:
insert_query = """
LET data = [
    { "nombre": "Roberto", "apellido": "Ramirez", "vivo": false, "caracteres": ["A","H","C"] },
    { "nombre": "Jaime", "apellido": "Quinteros", "vivo": true, "edad": 36, "caracteres": ["A","F","B"] },
    { "nombre": "Catalina", "apellido": "rojas", "vivo": false, "edad": 40, "caracteres": ["D","H","C"] },
    { "nombre": "Cesar", "apellido": "Lanaro", "vivo": true, "edad": 36, "caracteres": ["H","E","F"] },
    { "nombre": "David", "apellido": "Tardon", "vivo": true, "edad": 16, "caracteres": ["D","H","C"] },
    { "nombre": "Jose", "apellido": "Mora", "vivo": false, "caracteres": ["A","B","C","F"] },
    { "nombre": "Pedro", "apellido": "Baesa", "vivo": false, "caracteres": ["E","G","F"] },
    { "nombre": "Guillermo", "apellido": "Toro", "vivo": false, "caracteres": ["O","L","N"] },
    { "nombre": "Juan", "apellido": "Niebla", "vivo": true, "edad": 16, "caracteres": ["A","B","C","F"] },
    { "nombre": "Sara", "apellido": "Rojas", "vivo": true, "edad": 13, "caracteres": ["D","I","J"] },
    { "nombre": "Ariela", "apellido": "Araya", "vivo": true, "edad": 11, "caracteres": ["C","K","L"] },
    { "nombre": "Roberto", "apellido": "Cabezas", "vivo": false, "caracteres": ["A","B","C","K"] },
    { "nombre": "Theon", "apellido": "Greyjoy", "vivo": true, "edad": 16, "caracteres": ["E","R","K"] },
    { "nombre": "Bran", "apellido": "Stark", "vivo": true, "edad": 10, "caracteres": ["L","J"] },
    { "nombre": "Joffrey", "apellido": "Baratheon", "vivo": false, "edad": 19, "caracteres": ["I","L","O"] },
    { "nombre": "Sandor", "apellido": "Clegane", "vivo": true, "caracteres": ["A","P","K","F"] },
    { "nombre": "Tyrion", "apellido": "Lannister", "vivo": true, "edad": 32, "caracteres": ["F","K","M","N"] },
    { "nombre": "Khal", "apellido": "Drogo", "vivo": false, "caracteres": ["A","C","O","P"] },
    { "nombre": "Tywin", "apellido": "Lannister", "vivo": false, "caracteres": ["O","M","H","F"] },
    { "nombre": "Davos", "apellido": "Seaworth", "vivo": true, "edad": 49, "caracteres": ["C","K","P","F"] },
    { "nombre": "Samwell", "apellido": "Tarly", "vivo": true, "edad": 17, "caracteres": ["C","L","I"] },
    { "nombre": "Stannis", "apellido": "Baratheon", "vivo": false, "caracteres": ["H","O","P","M"] },
    { "nombre": "Melisandre", "vivo": true, "caracteres": ["G","E","H"] },
    { "nombre": "Margaery", "apellido": "Tyrell", "vivo": false, "caracteres": ["M","D","B"] },
    { "nombre": "Jeor", "apellido": "Mormont", "vivo": false, "caracteres": ["C","H","M","P"] },
    { "nombre": "Bronn", "vivo": true, "caracteres": ["K","E","C"] },
    { "nombre": "Varys", "vivo": true, "caracteres": ["M","F","N","E"] },
    { "nombre": "Shae", "vivo": false, "caracteres": ["M","D","G"] },
    { "nombre": "Talisa", "apellido": "Maegyr", "vivo": false, "caracteres": ["D","C","B"] },
    { "nombre": "Gendry", "vivo": false, "caracteres": ["K","C","A"] },
    { "nombre": "Ygritte", "vivo": false, "caracteres": ["A","P","K"] },
    { "nombre": "Tormund", "apellido": "Giantsbane", "vivo": true, "caracteres": ["C","P","A","I"] },
    { "nombre": "Gilly", "vivo": true, "caracteres": ["L","J"] },
    { "nombre": "Brienne", "apellido": "Tarth", "vivo": true, "edad": 32, "caracteres": ["P","C","A","K"] },
    { "nombre": "Ramsay", "apellido": "Bolton", "vivo": true, "caracteres": ["E","O","G","A"] },
    { "nombre": "Ellaria", "apellido": "Sand", "vivo": true, "caracteres": ["P","O","A","E"] },
    { "nombre": "Daario", "apellido": "Naharis", "vivo": true, "caracteres": ["K","P","A"] },
    { "nombre": "Missandei", "vivo": true, "caracteres": ["D","L","C","M"] },
    { "nombre": "Tommen", "apellido": "Baratheon", "vivo": true, "caracteres": ["I","L","B"] },
    { "nombre": "Jaqen", "apellido": "H'ghar", "vivo": true, "caracteres": ["H","F","K"] },
    { "nombre": "Roose", "apellido": "Bolton", "vivo": true, "caracteres": ["H","E","F","A"] },
    { "nombre": "The High Sparrow", "vivo": true, "caracteres": ["H","M","F","O"] }
]

FOR d IN data
    INSERT d INTO Personajes
"""
# python-arango
aql.execute(insert_query)

<Cursor>

In [ ]:
todos_personajes_nombre = """
FOR c IN Personajes
    RETURN { n:c.nombre,e:c.edad, v:c.vivo }
"""
# python-arango
query_result = aql.execute(todos_personajes_nombre)
for doc in  query_result:
  print(doc)

# Actualizando Documentos

In [ ]:
revivir_todos_personajes= """
FOR c IN Personajes
    FILTER c.vivo == False
    RETURN c._key
"""

neds_document_key = list()

# python-arango
query_result = aql.execute(revivir_todos_personajes)

for doc in  query_result:
    print("_key: " + str(doc))
    neds_document_key.append(doc)

In [12]:
neds_document_key[0]

'266211393669'



actualizando un documento por key:

In [ ]:
muertos = """
UPDATE @key
WITH { vivo: True}
IN Personajes
"""
bindVars = {'key': neds_document_key[0]}

aql.execute(muertos, batch_size=1, bind_vars=bindVars)

todos_personajes = """
FOR c IN Personajes
  RETURN { k:c._key, n:c.nombre,e:c.edad, v:c.vivo }
"""
query_result = aql.execute(todos_personajes)
for doc in  query_result:
    print(doc)

# Usando `FOR` para actualizar todos los personajes:

In [ ]:
actualizar_equipo_vivo = """
FOR c IN Personajes
    UPDATE c WITH { vivo: True, equipo : "Colo Colo" } IN Personajes
"""
# python-arango
aql.execute(actualizar_equipo_vivo)

todos_personajes_equipo_vivo = """
FOR c IN Personajes
    RETURN {"Nombre" : c.nombre, "Equipo" : c.equipo, "Vivo": c.vivo}
"""
# python-arango
query_result = aql.execute(todos_personajes_equipo_vivo)

for doc in  query_result:
    print(doc)

# Borrando Documentos por clave

Para eliminar documentos de una coleccion podemos utilizar REMOVE

In [15]:
buscando_alex= """
FOR c IN Personajes
    FILTER c.nombre == "Alex"
    RETURN c._key
"""
# python-arango
query_result = aql.execute(buscando_alex)

id_alex = None
for doc in  query_result:
    print(doc)
    id_alex=doc

266211393648


In [16]:
remove_alex = """
REMOVE @key IN Personajes
"""
bindVars = {'key': id_alex}

try:
    # python-arango
    aql.execute(remove_alex, bind_vars=bindVars)
except:
    print("Alex fue borrado.")

encontrar_alex = """
FOR c IN Personajes
    FILTER c.nombre == "Alex"
    RETURN c
"""
# python-arango
query_result = aql.execute(encontrar_alex, count=True)

if len(query_result) == 0 :
    print("Alex no fue encontrado.")

Alex no fue encontrado.


Con un FOR podemos eliminar todos los documentos de la coleccion:

In [17]:
borrar_todos = """
FOR c IN Personajes
    REMOVE c IN Personajes
"""
# python-arango
aql.execute(borrar_todos)

todos_personajes = """
FOR c IN Personajes
    RETURN c
"""
# python-arango
query_result = aql.execute(todos_personajes, count=True)

if len(query_result) == 0 :
    print("Todos los personajes fueron borrados.")

Todos los personajes fueron borrados.


In [18]:
filtro_edad = """
FOR c IN Personajes
    FILTER c.edad < 20 AND c.edad != null
    RETURN { nombre: c.nombre, edad: c.edad }
"""
# python-arango
query_result = aql.execute(filtro_edad)

for doc in  query_result:
    print(doc)

In [19]:
sort_nombre = """
FOR c IN Personajes
    SORT c.nombre
    RETURN c
"""
query_result = aql.execute(sort_nombre)

for doc in  query_result:
    print(doc)

In [20]:
sort_nombre = """
FOR c IN Personajes
    SORT c.nombre DESC
    RETURN c
"""
query_result = aql.execute(sort_nombre)

for doc in  query_result:
    print(doc)

In [21]:
sort_nombre = """
FOR c IN Personajes
    SORT c.nombre
    LIMIT 10
    RETURN c.nombre
"""

query_result = aql.execute(sort_nombre)

for doc in  query_result:
    print(doc)

In [22]:
pyAr_db.createCollection(name="caracteres")

ArangoDB collection name: caracteres, id: 398191583, type: document, status: loaded

In [23]:
insert_query = """
LET data = [
    { "_key": "A", "en": "strong", "es": "fuerte" },
    { "_key": "B", "en": "polite", "es": "polite" },
    { "_key": "C", "en": "loyal", "es": "leal" },
    { "_key": "D", "en": "beautiful", "es": "bonito" },
    { "_key": "E", "en": "sneaky", "es": "insidioso" },
    { "_key": "F", "en": "experienced", "es": "experimentado" },
    { "_key": "G", "en": "corrupt", "es": "corrupto" },
    { "_key": "H", "en": "powerful", "es": "potente" },
    { "_key": "I", "en": "naive", "es": "ingenuo" },
    { "_key": "J", "en": "unmarried", "es": "soltero" },
    { "_key": "K", "en": "skillful", "es": "habil" },
    { "_key": "L", "en": "young", "es": "joven" },
    { "_key": "M", "en": "smart", "es": "inteligente" },
    { "_key": "N", "en": "rational", "es": "racional" },
    { "_key": "O", "en": "ruthless", "es": "implacable" },
    { "_key": "P", "en": "brave", "es": "valiente" },
    { "_key": "Q", "en": "mighty", "es": "poderoso" },
    { "_key": "R", "en": "weak", "es": "debil" }
]


FOR d IN data
    INSERT d INTO caracteres
"""
# python-arango
aql.execute(insert_query)


<Cursor>

In [24]:
listar_todos = """
FOR c IN caracteres
    RETURN c
"""
# python-arango
query_result = aql.execute(listar_todos)

# pyArango
# query_result = db.AQLQuery(all_characters_names, rawResults=True)
for doc in  query_result:
    print(doc)

{'_key': 'A', '_id': 'caracteres/A', '_rev': '_gJ0Qszq---', 'en': 'strong', 'es': 'fuerte'}
{'_key': 'B', '_id': 'caracteres/B', '_rev': '_gJ0Qszq--_', 'en': 'polite', 'es': 'polite'}
{'_key': 'C', '_id': 'caracteres/C', '_rev': '_gJ0Qszq--A', 'en': 'loyal', 'es': 'leal'}
{'_key': 'D', '_id': 'caracteres/D', '_rev': '_gJ0Qszq--B', 'en': 'beautiful', 'es': 'bonito'}
{'_key': 'E', '_id': 'caracteres/E', '_rev': '_gJ0Qszq--C', 'en': 'sneaky', 'es': 'insidioso'}
{'_key': 'F', '_id': 'caracteres/F', '_rev': '_gJ0Qszq--D', 'en': 'experienced', 'es': 'experimentado'}
{'_key': 'G', '_id': 'caracteres/G', '_rev': '_gJ0Qszq--E', 'en': 'corrupt', 'es': 'corrupto'}
{'_key': 'H', '_id': 'caracteres/H', '_rev': '_gJ0Qszq--F', 'en': 'powerful', 'es': 'potente'}
{'_key': 'I', '_id': 'caracteres/I', '_rev': '_gJ0Qszq--G', 'en': 'naive', 'es': 'ingenuo'}
{'_key': 'J', '_id': 'caracteres/J', '_rev': '_gJ0Qszq--H', 'en': 'unmarried', 'es': 'soltero'}
{'_key': 'K', '_id': 'caracteres/K', '_rev': '_gJ0Qszq-

In [25]:
listar_personajes_caracteres = """
FOR c IN Personajes
    RETURN {nombre:c.nombre, caracteres:c.caracteres}
"""
query_result = aql.execute(listar_personajes_caracteres)
for doc in  query_result:
    print(doc)

# Ejemplos Joins

In [ ]:
#La función DOCUMENT() se puede utilizar para buscar uno o varios documentos a través de identificadores de documentos.
listar_personajes_caracteres = """
FOR c IN Personajes
    LIMIT 5
    RETURN {nombre:c.nombre, caracteres:DOCUMENT("caracteres", c.caracteres)}
"""
# python-arango
query_result = aql.execute(listar_personajes_caracteres)

for doc in  query_result:
    print(doc)

In [ ]:
listar_personajes_caracteres = """
FOR c IN Personajes
    LIMIT 5
    RETURN {nombre:c.nombre, caracteres:DOCUMENT("caracteres", c.caracteres)[*].es}
"""
# python-arango
query_result = aql.execute(listar_personajes_caracteres)

for doc in  query_result:
    print(doc)

In [ ]:
listar_personajes_caracteres = """
FOR c IN Personajes
    LIMIT 5
    RETURN {nombre:c.nombre, caracteres:DOCUMENT("caracteres", c.caracteres)[*].en}
"""
# python-arango
query_result = aql.execute(listar_personajes_caracteres)

for doc in  query_result:
    print(doc)

In [ ]:
todos_personajes_caracter = """
FOR c IN Personajes
    LIMIT 5
    RETURN MERGE(c, { caracter: DOCUMENT("caracteres", c.caracteres)[*].es } )
"""
query_result = aql.execute(todos_personajes_caracter)
for doc in  query_result:
    print(doc)

In [ ]:
all_characters_traits = """
FOR c IN Personajes
  LIMIT 10
  SORT c.apellido
  RETURN MERGE(c, {
    caracter: (
      FOR key IN c.caracteres
        FOR t IN caracteres
          FILTER t._key == key
          RETURN t.es
    )
  })
"""
# python-arango
query_result = aql.execute(all_characters_traits)

for doc in  query_result:
    print(doc)

* https://www.arangodb.com/docs/stable/aql/tutorial.html